<br>
<br>
<br>

# Calculate Window Centroid 

<br>
<br>
<br>

<img src="https://github.com/KarolinaPntzt/window_centroid/blob/main/img/window_centroids.PNG?raw=True" width=70%></img>

<br>
<br>
<br>

This notebook is dedicated to computing the centroid of rectangular windows on vertical facades of 3D buildings. The same code can be used to compute the centroid of any rectangular 2D geometry placed on a vertical 3D surface. 

<u><b>Requirements:</b></u>

- An active ESRI license for ArcGIS Pro 2.8 or later (must include rights to use 3D functions)
- Shapefile with rectangular geometries placed on vertical surfaces


<br>
<br>
<br>

The notebook is organized as follows:
- [Help functions](#help_funcs)
- [Window centroid function - Main](#win_centroid_func)
- [Set input and output paths](#set_input_info)
- [Execute computations](#execute_computations)
- [Control output](#control_output)
- [Reference links](#ref_links)

<br>
<br>
<br>

In [ ]:
#Import modules:
import arcpy
import os

<a id="help_funcs"></a>
<br>
<br>

## Help functions

In [ ]:
#Function that takes the name of a shapefile stored in the 
#current working environment as input (string) and returns
#the column names of the shapefile:
def get_field_names(fc):
    
    #Get a list of fieldnames for input shapefile:
    f_names = [f.name for f in arcpy.ListFields(fc)]
    
    #Return list with fieldnames:
    return f_names

In [ ]:
#Function that takes the name of a shapefile stored in the 
#current working environment as input (string) and returns
#the content of the corresponding Table Of Contents (TOC):
def get_TOC(fc):
    
    #Get a list of field-names:
    f_names = [f.name for f in arcpy.ListFields(fc)]
    
    #Open shapefile and print every row in TOC:
    with arcpy.da.SearchCursor(fc, f_names) as cursor:
        
        for row in cursor:
            
            print(row)

In [ ]:
#Function that takes the name of a shapefile stored in the 
#current working environment as input (string) together with 
#the column name whose values we want to extract and a boolean 
#(unique=True returns only unique values in column) and returns
#the content of the corresponding column in a list:
def get_TOC_column(fc, colname, unique):
    
    #Create a list to store the column values:
    ls = []
    
    #Open shapefile:
    with arcpy.da.SearchCursor(fc, colname) as cursor:
        
        #Loop through every row in shapefile:
        for row in cursor:
            
            #If user wants the output list to only include unique values:
            if (unique):
                
                #Check that values is not already stored in the list:
                if row[0] not in ls:
                    
                    #Append value:
                    ls.append(row[0])
                    
            else:
                
                #Append column value to list:
                ls.append(row[0])
    
    #return list:
    return ls

In [ ]:
#Function that creates a new numeric field/column for a given feature class> 
#(feature_class-string, colname-string, datatype-string ("LONG"), numofvalues-integer, numofdecvalues-integer)
def create_long_field(fc, colname, datatype, numofvalues, numofdecvalues):
    
    #Extract field-names from shapefile:
    fieldnames = [f.name for f in arcpy.ListFields(fc)]
    
    #Check if "NN_dist" field already exists:
    if(colname not in fieldnames):

        #Add new field (distance to nearest neighbor) to shapefile: 
        try:
            arcpy.AddField_management(fc, colname, datatype, numofvalues, numofdecvalues)
            
        #Print exception error, if field cannot be added:
        except Exception as e:
            print("Error: " + e.args[0])
    
    
    

In [ ]:
#Function that reads and returns the name of the 
#spatial reference system for the input feature layer:
def get_CRS(fc):
    
    #Extract spatial reference system information from feature class:
    spatial_ref = arcpy.Describe(fc).spatialReference
    
    #Return:
    return spatial_ref.name
    

<a id="win_centroid_func"></a>

<br>
<br>

## Compute window centroids - (main function)

In [ ]:
#Function that takes a multipatch layer of rectangular windows as input 
#and returns a new feature class with their centroids (point layer):
def compute_w_centroid(fc, vcs, out_file):

    #Add fields in window shapefile to store min/max x/y/z coordinates for every window:
    create_long_field(fc, "X_MIN", "LONG", 9, 2)
    create_long_field(fc, "Y_MIN", "LONG", 9, 2)
    create_long_field(fc, "Z_MIN", "LONG", 9, 2)
    create_long_field(fc, "X_MAX", "LONG", 9, 2)
    create_long_field(fc, "Y_MAX", "LONG", 9, 2)
    create_long_field(fc, "Z_MAX", "LONG", 9, 2)

    #Add fields in output shapefile to store window-centroid x/y/z-coordinates:
    create_long_field(fc, "C_X_coord", "LONG", 9, 2)
    create_long_field(fc, "C_Y_coord", "LONG", 9, 2)
    create_long_field(fc, "C_Z_coord", "LONG", 9, 2)

    #Get geometry information on min/max x-/y-/z-coordinates for every window:
    arcpy.management.CalculateGeometryAttributes(fc, [["X_MIN", "EXTENT_MIN_X"],
                                                      ["Y_MIN", "EXTENT_MIN_Y"],
                                                      ["Z_MIN", "EXTENT_MIN_Z"],
                                                      ["X_MAX", "EXTENT_MAX_X"],
                                                      ["Y_MAX", "EXTENT_MAX_Y"],
                                                      ["Z_MAX", "EXTENT_MAX_Z"]])


    #Calculate the centroid x-/y-/z-coordinates for every window:
    arcpy.CalculateField_management(fc, "C_X_coord", "!X_MIN!+((!X_MAX!-!X_MIN!)/2)", "PYTHON_9.3", "")
    arcpy.CalculateField_management(fc, "C_Y_coord", "!Y_MIN!+((!Y_MAX!-!Y_MIN!)/2)", "PYTHON_9.3", "")
    arcpy.CalculateField_management(fc, "C_Z_coord", "!Z_MIN!+((!Z_MAX!-!Z_MIN!)/2)", "PYTHON_9.3", "")


    #Create separate shapefile (point layer) with window centroids:
    arcpy.management.XYTableToPoint(fc, out_file,
                                    x_field = "C_X_coord", 
                                    y_field = "C_Y_coord",
                                    z_field = "C_Z_coord",
                                    coordinate_system = arcpy.SpatialReference(arcpy.Describe(fc).spatialReference.factoryCode,vcs))

    #Delete columns from original window shapefile:
    arcpy.DeleteField_management(fc, 
                                 ["X_MIN", "Y_MIN", "Z_MIN",
                                  "X_MAX", "Y_MAX", "Z_MAX", 
                                  "C_X_coord", "C_Y_coord", "C_Z_coord"])

    #Delete columns from output window-centroid shapefile:
    arcpy.DeleteField_management(out_file, 
                                 ["X_MIN", "Y_MIN", "Z_MIN",
                                  "X_MAX", "Y_MAX", "Z_MAX"])

<a id="set_input_info"></a>
<br>
<br>

## Set input and output location information

<br>
<br>

In [ ]:
#Set input variables:
building_number = 8
working_environment = r"C:\Users\Username\path_to_window_shapefile"
input_feature_class = "n"+str(building_number)+"_w.shp"
VCS = 5613 #vertical coordinate system (5613 corresponds to RH2000) 

#Set environment settings:
arcpy.env.workspace = working_environment

#Set output variables:
out_path = "C:\\Users\\Username\\output_folder\\"
out_file = "boi"+str(building_number)+"_w_centroids.shp"
output = out_path+out_file

In [ ]:
# Get description of input feature class:
desc = arcpy.da.Describe(input_feature_class)

<a id="execute_computations"></a>
<br>
<br>

## Execute calculations

<br>
<br>

In [ ]:
#Call function to compute window centroids:
compute_w_centroid(input_feature_class, VCS, output)

<a id="control_output"></a>
<br>
<br>

## Conntrol output

In [ ]:
#Check columns of output file (window centroids):
get_field_names(output)

In [ ]:
#Check content of output file attribute table (window centroids): 
get_TOC(output)

In [ ]:
#Check columns of input file (windows):
get_field_names(input_feature_class)

In [ ]:
#Check content of input file (windows) attribute table: 
get_TOC(input_feature_class)

<a id="ref_links"></a>

<br>
<br>

## Reference links:

- Extensions:
    - https://gis.stackexchange.com/questions/54044/arcgis-error-000824-the-tool-is-not-licensed-in-arcpy
    - https://pro.arcgis.com/en/pro-app/2.8/arcpy/functions/checkextension.htm

    
- Spatial Reference:
    - https://pro.arcgis.com/en/pro-app/2.8/arcpy/classes/spatialreference.htm
    

- Calculate Geometry Attributes:
    - https://pro.arcgis.com/en/pro-app/3.0/tool-reference/data-management/calculate-geometry-attributes.htm
    
    
- Calculate Field:
    - https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/calculate-field.htm


- XY Table To Point:
    - https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/xy-table-to-point.htm


- Delete Shapefile:
    - https://pro.arcgis.com/en/pro-app/2.8/tool-reference/data-management/delete.htm
    
    
- Find files in directory:
    - https://stackoverflow.com/questions/3964681/find-all-files-in-a-directory-with-extension-txt-in-python
   


<br>
<br>